In [2]:
import urllib.request
import json
import pprint
import sys
import _thread
import websocket


In [3]:
password='Cooper2e'

In [4]:
def get_token(password):
    obj = { 'APIPassword': password}
    json_data = json.dumps(obj).encode('utf8')
    url = 'http://localhost:18081/kabusapi/token' #検証用
    #url = 'http://localhost:18080/kabusapi/token' #本番用
    req = urllib.request.Request(url, json_data, method='POST')
    req.add_header('Content-Type', 'application/json')

    try:
        with urllib.request.urlopen(req) as res:
            print(res.status, res.reason)
            for header in res.getheaders():
                print(header)
            print()
            content = json.loads(res.read())
            pprint.pprint(content)
            return content["Token"]
    except urllib.error.HTTPError as e:
        print(e)
        content = json.loads(e.read())
        pprint.pprint(content)
    except Exception as e:
        print(e)
        

In [5]:
token=get_token("Cooper2e")

200 OK
('Content-Length', '59')
('Content-Type', 'application/json; charset=utf-8')
('Server', 'Microsoft-HTTPAPI/2.0')
('X-Content-Type-Options', 'nosniff')
('X-XSS-Protection', '1; mode=block')
('X-Frame-Options', 'DENY')
('Content-Security-Policy', "default-src 'none'")
('Access-Control-Allow-Origin', '*')
('Date', 'Fri, 29 Jan 2021 14:38:33 GMT')
('Connection', 'close')

{'ResultCode': 0, 'Token': '9e988a00e3124c2cbb461d0522dab17d'}


# 1．注文発注（現物）
現物買い、現物売り

In [12]:
#現物買い

'''
現物と信用の売買コードは同じでパラーメータを変更するだけ
'Password'       注文パスワード:
'Symbol'         銘柄コード:
'Exchange'       市場コード:１東証、3名証,5福証,6札証
'SecurityType'   商品識別:1株式
'FrontOrderType' 執行条件10、13～17成行、20～27指値（10 or 20）
 'Side'          売買区分,1売り,2買い
 'CashMargin'    信用区分:1現物、２新規、３返済
 'DelivType'     受渡区分:0指定なし、1自動振替、2預り金｛現物買い指定必須/現物売り0指定｝
 'FundType'　　　資産区分（預り区分）:'半角2つ'現物売り/02保護/AA信用代用/11信用取引
 'AccountType'   口座識別:2一般/4特定/12法人
 'Qty'　　　　　 注文数量
 'Price'　　　　 注文価格
 'ExpireDay'　　 注文有効期限：0本日/yyyyMMdd形式
 　
 その他
 'ClosePositionOrder' 決済順序:0～７（損益（高/低)/日付（新/古））、信用取引必須
 'ClosePositions'     返済建玉指定
 詳しくはネットで
'''
sym='9433'
qty=100
price=3000
def sendorder_buy(sym,qty,price):
    obj = { 'Password': password,
            'Symbol': sym,      
            'Exchange': 1,         
            'SecurityType': 1,     
            'FrontOrderType': 20,
            'Side': '2',           
            'CashMargin': 1,       
            'DelivType': 2,        
            'FundType': 'AA',
            'AccountType': 4,
            'Qty': qty,
            'Price': price,
            'ExpireDay': 0 }
    json_data = json.dumps(obj).encode('utf-8')

    url = 'http://localhost:18081/kabusapi/sendorder'
    req = urllib.request.Request(url, json_data, method='POST')
    req.add_header('Content-Type', 'application/json')
    req.add_header('X-API-KEY', token)

    try:
        with urllib.request.urlopen(req) as res:
            print(res.status, res.reason)
            for header in res.getheaders():
                print(header)
            print()
            content = json.loads(res.read())
            pprint.pprint(content)
            return content
    except urllib.error.HTTPError as e:
        print(e)
        content = json.loads(e.read())
        pprint.pprint(content)
    except Exception as e:
        print(e)

sendorder(sym,qty,price)

200 OK
('Content-Length', '27')
('Content-Type', 'application/json; charset=utf-8')
('Server', 'Microsoft-HTTPAPI/2.0')
('X-Content-Type-Options', 'nosniff')
('X-XSS-Protection', '1; mode=block')
('X-Frame-Options', 'DENY')
('Content-Security-Policy', "default-src 'none'")
('Access-Control-Allow-Origin', '*')
('Date', 'Fri, 29 Jan 2021 14:43:46 GMT')
('Connection', 'close')

{'OrderId': None, 'Result': 0}


{'Result': 0, 'OrderId': None}

In [15]:
#現物売り
def sendorder_sell(sym,qty,price):
    obj = { 'Password': '123456',
            'Symbol': sym,
            'Exchange': 1,
            'SecurityType': 1,
            'FrontOrderType': 20,
            'Side': '1',
            'CashMargin': 1,
            'DelivType': 0,
            'FundType': '  ',
            'AccountType': 2,
            'Qty': qty,
            'Price': price,
            'ExpireDay': 20200924 }
    json_data = json.dumps(obj).encode('utf-8')

    url = 'http://localhost:18080/kabusapi/sendorder'
    req = urllib.request.Request(url, json_data, method='POST')
    req.add_header('Content-Type', 'application/json')
    req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

    try:
        with urllib.request.urlopen(req) as res:
            print(res.status, res.reason)
            for header in res.getheaders():
                print(header)
            print()
            content = json.loads(res.read())
            pprint.pprint(content)
            return content
    except urllib.error.HTTPError as e:
        print(e)
        content = json.loads(e.read())
        pprint.pprint(content)
    except Exception as e:
        print(e)
sendorder_sell(sym,qty,price)

HTTP Error 401: Unauthorized
{'Code': 4001009, 'Message': 'APIキー不一致'}


# 5．注文取消 コマンド

In [19]:
def cancelorder(OID):
    obj = { 'OrderID': OID, 'Password': password }
    json_data = json.dumps(obj).encode('utf8')

    url = 'http://localhost:18081/kabusapi/cancelorder'
    req = urllib.request.Request(url, json_data, method='PUT')
    req.add_header('Content-Type', 'application/json')
    req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

    try:
        with urllib.request.urlopen(req) as res:
            print(res.status, res.reason)
            for header in res.getheaders():
                print(header)
            print()
            content = json.loads(res.read())
            pprint.pprint(content)
            return content
    except urllib.error.HTTPError as e:
        print(e)
        content = json.loads(e.read())
        pprint.pprint(content)
    except Exception as e:
        print(e)


In [20]:
cancelorder(OID)

NameError: name 'OID' is not defined

# 2．注文発注（信用）
新規、返済（決済順序）、返済（返済建玉指定）

In [ ]:
#新規
obj = { 'Password': '123456',
        'Symbol': '9433',
        'Exchange': 1,
        'SecurityType': 1,
        'FrontOrderType': 20,
        'Side': '2',
        'CashMargin': 2,
        'MarginTradeType': 2,
        'DelivType': 0,
        'AccountType': 2,
        'Qty': 100,
        'Price': 2762.5,
        'ExpireDay': 20200924 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)
#返済（決済順序）
obj = { 'Password': '123456',
        'Symbol': '9433',
        'Exchange': 1,
        'SecurityType': 1,
        'FrontOrderType': 20,
        'Side': '1',
        'CashMargin': 3,
        'MarginTradeType': 2,
        'DelivType': 2,
        'AccountType': 2,
        'Qty': 100,
        'ClosePositionOrder': 1,
        'Price': 2762.5,
        'ExpireDay': 20200924 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)
#返済（返済建玉指定）
obj = { 'Password': '123456',
        'Symbol': '9433',
        'Exchange': 1,
        'SecurityType': 1,
        'FrontOrderType': 20,
        'Side': '1',
        'CashMargin': 3,
        'MarginTradeType': 1,
        'DelivType': 2,
        'AccountType': 4,
        'Qty': 200,
        'ClosePositions': [{'HoldID':'E20200924*****','Qty':100},{'HoldID':'E20200924*****','Qty':100}],
        'Price': 2762.5,
        'ExpireDay': 20200924 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)

# 3．注文発注（先物）
新規、返済（決済順序）、返済（返済建玉指定）

In [ ]:
#新規
obj = { 'Password': '123456',
        'Symbol': '165120018',
        'Exchange': 23,
        'TradeType': 1,
        'TimeInForce': 1,
        'Side': '1',
        'Qty': 3,
        'Price': 22000,
        'ExpireDay': 20200925,
        'FrontOrderType': 20 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder/future'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)
#返済（決済順序）
obj = { 'Password': '123456',
        'Symbol': '165120018',
        'Exchange': 23,
        'TradeType': 2,
        'TimeInForce': 1,
        'Side': '2',
        'Qty': 3,
        'ClosePositionOrder': 1,
        'Price': 22000,
        'ExpireDay': 20200925,
        'FrontOrderType': 20 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder/future'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)
#返済（返済建玉指定）
obj = { 'Password': '123456',
        'Symbol': '165120018',
        'Exchange': 23,
        'TradeType': 2,
        'TimeInForce': 1,
        'Side': '2',
        'Qty': 3,
        'ClosePositions': [{'HoldID':'E20200924*****','Qty':2},{'HoldID':'E20200924*****','Qty':1}],
        'Price': 22000,
        'ExpireDay': 20200925,
        'FrontOrderType': 20 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder/future'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)

# 4．注文発注（OP）
新規、返済（決済順序）、返済（返済建玉指定）

In [ ]:
#新規
obj = { 'Password': '123456',
        'Symbol': '145123218',
        'Exchange': 23,
        'TradeType': 1,
        'TimeInForce': 2,
        'Side': '1',
        'Qty': 5,
        'Price': 0,
        'ExpireDay': 20200924,
        'FrontOrderType': 120 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder/option'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)
#返済（決済順序）
obj = { 'Password': '123456',
        'Symbol': '145123218',
        'Exchange': 23,
        'TradeType': 2,
        'TimeInForce': 2,
        'Side': '2',
        'Qty': 1,
        'ClosePositionOrder': 1,
        'Price': 0,
        'ExpireDay': 20200904,
        'FrontOrderType': 120 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder/option'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)
#返済（返済建玉指定）
obj = { 'Password': '123456',
        'Symbol': '145123218',
        'Exchange': 23,
        'TradeType': 2,
        'TimeInForce': 2,
        'Side': '2',
        'Qty': 3,
        'ClosePositions': [{'HoldID':'E20200924*****','Qty':2},{'HoldID':'E20200924*****','Qty':1}],
        'Price': 0,
        'ExpireDay': 20200904,
        'FrontOrderType': 120 }
json_data = json.dumps(obj).encode('utf-8')

url = 'http://localhost:18080/kabusapi/sendorder/option'
req = urllib.request.Request(url, json_data, method='POST')
req.add_header('Content-Type', 'application/json')
req.add_header('X-API-KEY', 'ed94b0d34f9441c3931621e55230e402')

try:
    with urllib.request.urlopen(req) as res:
        print(res.status, res.reason)
        for header in res.getheaders():
            print(header)
        print()
        content = json.loads(res.read())
        pprint.pprint(content)
except urllib.error.HTTPError as e:
    print(e)
    content = json.loads(e.read())
    pprint.pprint(content)
except Exception as e:
    print(e)